In [16]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

# Importing custom libraries
import sys
sys.path.append('/utilities/')
from utilities.data_manipulation import pivot_dataframe, convert_to_supervised, rename_dataframe_supervised, plot_results, plot_comparison

# Random seed for reproducibility
tf.random.set_seed(42)


### Loading Data

In [19]:
ids = ['meantemp', 'humidity']
formatted_df = pd.read_csv('Data/DailyDelhiClimate.csv')
formatted_df.head(10)

,Date,meantemp,humidity
0,1/1/2013,10.000000,84.500000
1,1/2/2013,7.400000,92.000000
2,1/3/2013,7.166667,87.000000
3,1/4/2013,8.666667,71.333333
4,1/5/2013,6.000000,86.833333
5,1/6/2013,7.000000,82.800000
6,1/7/2013,7.000000,78.600000
7,1/8/2013,8.857143,63.714286
8,1/9/2013,14.000000,51.250000
9,1/10/2013,11.000000,62.000000


### Graphing Initial Dataset

In [22]:
# Create traces
fig = go.Figure()

for factor_level in ids:
    # Adding plot of original_df
    fig.add_trace(go.Scatter(x=formatted_df['Date'], y=formatted_df[factor_level],
                        mode='lines',
                        name=factor_level))
fig.update_layout(title = "Temp and Humidity over Time, Delhi India")

### First Trying with Just One Series

In [4]:
# creating scalar
scaler = MinMaxScaler(feature_range=(0, 1))

# Normalizing target columns
for col in ids:
    formatted_df [col] = scaler.fit_transform(formatted_df[[col]])
formatted_df.head(10)

,Unnamed: 0,1064,1067,Date
0,0,0.252174,0.397436,2024-08-14 05:00:00+00:00
1,1,0.269565,0.397436,2024-08-14 06:00:00+00:00
2,2,0.260870,0.423077,2024-08-14 07:00:00+00:00
3,3,0.269565,0.435897,2024-08-14 08:00:00+00:00
4,4,0.286957,0.435897,2024-08-14 09:00:00+00:00
5,5,0.321739,0.461538,2024-08-14 10:00:00+00:00
6,6,0.347826,0.474359,2024-08-14 11:00:00+00:00
7,7,0.321739,0.474359,2024-08-14 12:00:00+00:00
8,8,0.330435,0.500000,2024-08-14 13:00:00+00:00
9,9,0.339130,0.500000,2024-08-14 14:00:00+00:00


### Creating Training Set

In [5]:
# setting test/train ratio
total_observations = len(formatted_df)
train_ratio = 0.7

# performing time based test/train split
train_df = formatted_df[:int(total_observations * train_ratio)]
test_df = formatted_df[int(total_observations * train_ratio):]
print(len(train_df), len(test_df))
test_df.head(10)


140 60


,Unnamed: 0,1064,1067,Date
140,140,0.121739,0.205128,2024-08-20 01:00:00+00:00
141,141,0.147826,0.243590,2024-08-20 02:00:00+00:00
142,142,0.139130,0.217949,2024-08-20 03:00:00+00:00
143,143,0.173913,0.269231,2024-08-20 04:00:00+00:00
144,144,0.304348,0.230769,2024-08-20 05:00:00+00:00
145,145,0.269565,0.256410,2024-08-20 06:00:00+00:00
146,146,0.286957,0.256410,2024-08-20 07:00:00+00:00
147,147,0.269565,0.243590,2024-08-20 08:00:00+00:00
148,148,0.252174,0.256410,2024-08-20 09:00:00+00:00
149,149,0.243478,0.320513,2024-08-20 10:00:00+00:00


In [6]:
print(test_df.columns)

Index(['Unnamed: 0', '1064', '1067', 'Date'], dtype='object')


### Converting Data to Windowed Format

In [7]:
timesteps = 6
features = len(ids)

count = 0
# Storing results
train_x_all, test_x_all = np.array([]), np.array([])
train_y_all, test_y_all = np.array([]), np.array([])

# Iterating through each series of interest
for val in ids:

    train_vals, test_vals = train_df[val].values, test_df[val].values

    # Transforming to time series                                                                                                                                                       
    train_data, test_data = convert_to_supervised(train_vals, n_in=timesteps), convert_to_supervised(test_vals, n_in=timesteps)

    # Converting to dataframe
    train_df_sup, test_df_sup = rename_dataframe_supervised(pd.DataFrame(train_data)), rename_dataframe_supervised(pd.DataFrame(test_data))

    # Separating x and y
    train_y, train_x = train_df_sup['t'].to_numpy(), train_df_sup.drop(columns = ['t']).to_numpy()
    test_y, test_x = test_df_sup['t'].to_numpy(), test_df_sup.drop(columns = ['t']).to_numpy()

    if count == 0:
        train_x_all = train_x
        train_y_all = train_y
        test_x_all = test_x
        test_y_all = test_y
    else:
        train_x_all = np.concatenate((train_x_all, train_x), axis = 1)
        train_y_all = np.stack((train_y_all, train_y), axis = 0)
        test_x_all = np.concatenate((test_x_all, test_x), axis = 1)
        test_y_all = np.stack((test_y_all, test_y), axis = 0)

    count += 1
train_y_all = train_y_all.T
test_y_all = test_y_all.T
print('Y-Train Shape: ', train_y_all.shape)
print('X-Train Shape: ', train_x_all.shape)


Y-Train Shape:  (134, 2)
X-Train Shape:  (134, 12)


In [8]:
# reshape input to be [samples, time steps, features]
# otherwise known as [rows x timesteps into future x columns]
train_x_all = np.reshape(train_x_all, (train_x_all.shape[0], 1, train_x_all.shape[1]))
test_x_all = np.reshape(test_x_all, (test_x_all.shape[0], 1, test_x_all.shape[1]))


In [9]:
print('Train X shape:', train_x_all.shape)
print('Test X shape:', test_x_all.shape)

Train X shape: (134, 1, 12)
Test X shape: (54, 1, 12)


### Now we can Make a Model
First we define and two helper functions to train a generic keras model(will reuse this with an LSTM model and DNN) and a graphing function to evaluate results.

In [10]:
def train_and_predict(model, train_x_all, test_x_all, train_y_all, test_y_all, epochs):

    # compiling and fitting model
    model.compile(loss='mean_absolute_error', optimizer='adam')
    model.fit(train_x_all, train_y_all, epochs=epochs, batch_size=1, verbose=3)

    # make predictions
    trainPredict = model.predict(train_x_all).reshape(train_y_all.shape[0], features)
    testPredict = model.predict(test_x_all).reshape(test_y_all.shape[0], features)

    # inverting predictions back to initial scale
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform(train_y_all)
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform(test_y_all)

    return trainPredict, testPredict, trainY, testY 

def plot_comparison(train_pred, test_pred, train_y, test_y, dates, model_type):

    factor_levels = train_pred.shape[1]
    fig = make_subplots(rows=factor_levels, cols=1, subplot_titles=("Plot 1", "Plot 2"))
    mapes = []

    # iterating through factor levels
    for level in range(factor_levels):
        fig.layout.annotations[level].update(text =  "Updated Plot 1")
        # defining dates for x axis
        train_dates = dates[:train_pred.shape[0]]
        test_dates = dates[-test_pred.shape[0]:]
        fig.append_trace(go.Scatter(x=train_dates, y=train_y[:, level],
                            mode='lines',
                            name='Actual-Train'),
                            row=level+1, col=1)
        # test set
        fig.append_trace(go.Scatter(x=test_dates, y=test_y[:, level],
                            mode='lines',
                            name='Actual-Test'),
                            row=level+1, col=1)
        
        # train pred
        fig.append_trace(go.Scatter(x=train_dates, y=train_pred[:, level],
                            mode='markers',
                            name='Pred-Train'),
                            row=level+1, col=1)
        # test pred
        fig.append_trace(go.Scatter(x=test_dates, y=test_pred[:, level],
                            mode='markers',
                            name='Pred-Test'),
                            row=level+1, col=1)
        
        # calculating mape on test set and updating title
        mape = round(mean_absolute_percentage_error(test_y[:, level],test_pred[:, level]),4)
        fig.layout.annotations[level].update(text =  "Series {series}, Test MAPE, {mape}".format(mape = mape, series = level + 1))
        mapes.append(mape)
        
    fig.update_layout(height=600, width=600, title_text="{model_type}, Forecast By Series, Avg MAPE {mape}".format(model_type = model_type, mape = round(sum(mapes)/len(mapes), 4)))
    fig.show()

In [11]:
# defining lstm model
units = 8
model_lstm = Sequential()
model_lstm.add(LSTM(units, input_shape=(1, int(timesteps * features))))
model_lstm.add(Dense(features))
epochs = 100

# training model and returning predictions
trainPredict_lstm, testPredict_lstm, trainY, testY = train_and_predict(model_lstm, train_x_all, test_x_all, train_y_all, test_y_all, epochs)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

### Examining and Graphing Data

In [12]:
# graphing data
dates = formatted_df['Date'].sort_values(ascending=True).drop_duplicates().to_list()
model_type = 'LSTM'
plot_comparison(trainPredict_lstm, testPredict_lstm, trainY, testY, dates, model_type)

In [13]:
# defining DNN model
units = 16
model_dnn = Sequential()
model_dnn.add(Dense(units = units, activation = 'relu'))
model_dnn.add(Dense(units = units, activation = 'relu'))
model_dnn.add(Dense(features))
epochs = 100

# training model and returning predictions
trainPredict_dnn, testPredict_dnn, trainY, testY = train_and_predict(model_dnn, train_x_all, test_x_all, train_y_all, test_y_all, epochs)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [14]:
model_type = 'DNN'
plot_comparison(trainPredict_dnn, testPredict_dnn, trainY, testY, dates, model_type)

### Comparing to Naive Approach
The simplest approach would be predicting the future value is equal to the previous value. We'll do this and evaluate its performance.

In [15]:
base_df=  pd.read_csv('Data/pivoted_data.csv')
future_y = base_df['1064'].iloc[1:]
naive_pred = base_df['1064'].iloc[:-1]

# calculating mape with naive approach
naive_mape_1064 =  mean_absolute_percentage_error(future_y[-54:], naive_pred[-54:])
print('Test Score 1064: %.4f MAPE' % (naive_mape_1064))


future_y = base_df['1067'].iloc[1:]
naive_pred = base_df['1067'].iloc[:-1]

# calculating mape with naive approach
naive_mape_1067 =  mean_absolute_percentage_error(future_y[-54:], naive_pred[-54:])
print('Test Score 1067: %.4f MAPE' % (naive_mape_1067))
print('Avg Mape: ', round((naive_mape_1064 +  naive_mape_1067)/2, 4))


Test Score 1064: 0.0663 MAPE
Test Score 1067: 0.0800 MAPE
Avg Mape:  0.0732


Can see that LSTM performs just slightly better than our naive approach although the naive approach works surprisingly well in this case.